In [1]:
from PyQt5 import QtGui
import pyqtgraph as pg
import pandas as pd
import numpy as np

C:\Users\nshawen\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
## Always start by initializing Qt (only once per application)
app = QtGui.QApplication([])

In [5]:
class AccViewer(QtGui.QDialog):
    def __init__(self, parent=None):
        super(AccViewer,self).__init__(parent)
        
        self.WindowRange = 10 # time (s) to add to both sides of acc signal
        
        ## Load timestamp data
        Labels = pd.read_hdf('//FS2.smpp.local\RTO\CIS-PD MUSC\decoded_forms\\form509.h5')
        Questions = pd.read_sas('//FS2.smpp.local\RTO\CIS-PD MUSC\datadictionary.sas7bdat')
        Questions = Questions[Questions.zFormID == 509.0]
        Questions = Questions.drop(columns = list(set(Questions.columns)-set(['zFieldNm','zFieldNb'])))
        Questions.zFieldNm = Questions.zFieldNm.apply(
            lambda x: x.replace(b'\x92',b"'").replace(b'\x97',b'--').decode() if isinstance(x, bytes) else x)
        Questions.zFieldNb = Questions.zFieldNb.apply(
            lambda x: x.replace(b'\x92',b"'").replace(b'\x97',b'--').decode() if isinstance(x, bytes) else x)
        
        self.questions = Questions
        self.timestamps = Labels
        self.taskInd = 0
        self.taskList = np.array(['Shaking', 'Standing', 'Walking', 'Walking while counting', 'Finger to nose--right hand', 
                                  'Finger to nose--left hand', 'Alternating right hand movements', 
                                  'Alternating left hand movements', 'Sit to stand', 'Drawing on a paper', 
                                  'Typing on a computer keyboard', 'Assembling nuts and bolts', 'Taking a glass of water',
                                  'Organizing sheets in a folder', 'Folding towels', 'Sitting'])
        self.sessionInd = 0
        self.sessionList = np.array(['2 Weeks: Time 0', '2 Weeks: Time 30', '2 Weeks: Time 60', '2 Weeks: Time 90', 
                                     '2 Weeks: Time 120', '2 Weeks: Time 150', '1 Month'])
        self.loadedSubject = None
        
        ## Create some widgets to be placed inside
        self.plotBtn = QtGui.QPushButton('Plot Data')
        self.subjtxt = QtGui.QLineEdit('1004')
        self.sesstxt = QtGui.QLineEdit(self.sessionList[self.sessionInd])
        self.tasktxt = QtGui.QLineEdit(self.taskList[self.taskInd])
        self.nextBtn = QtGui.QPushButton('Next Task')
        self.saveBtn = QtGui.QPushButton('Save Data')
        self.listw = QtGui.QListWidget()
        self.plot = pg.PlotWidget()
        self.stLine = pg.InfiniteLine(pen=(4,5),movable=True)
        self.spLine = pg.InfiniteLine(pen=(5,5),movable=True)

        self.plotBtn.clicked.connect(self.plotBtn_clicked)
        self.nextBtn.clicked.connect(self.nextBtn_clicked)
        self.saveBtn.clicked.connect(self.saveBtn_clicked)
        self.sesstxt.editingFinished.connect(self.sessTxt_edited)
        self.tasktxt.editingFinished.connect(self.taskTxt_edited)
        

        ## Create a grid layout to manage the widgets size and position
        layout = QtGui.QGridLayout()

        ## Add widgets to the layout in their proper positions
        layout.addWidget(self.plotBtn, 0, 0)
        layout.addWidget(self.subjtxt, 1, 0)
        layout.addWidget(self.sesstxt, 2, 0)
        layout.addWidget(self.tasktxt, 3, 0)
        layout.addWidget(self.nextBtn, 4, 0)
        layout.addWidget(self.saveBtn, 5, 0)
        layout.addWidget(self.listw, 6, 0)  # list widget goes in bottom-left
        layout.addWidget(self.plot, 0, 1, 7, 1)  # plot goes on right side

        self.setLayout(layout)

    def plotBtn_clicked(self):
        sessInds = (self.timestamps.SubjectCode.astype(int) == int(self.subjtxt.text())
                   ) & (self.timestamps.VisitNm == self.sesstxt.text())
        
        watchTime = self.timestamps[sessInds]['Q146_UTC'].values[0]
        
        StQinds = self.questions.zFieldNm == (self.tasktxt.text() + ' start time')
        StQNb = self.questions[StQinds].zFieldNb
        stTime = self.timestamps[sessInds][StQNb].values[0][0]
        if type(stTime)!=str:
            print('No start time found for subject ' + self.subjtxt.text()
                  + ' ' + self.sesstxt.text() + ' ' + self.tasktxt.text())
            return
        stTime = watchTime + self.getTimeDelta(stTime)
        
        SpQinds = self.questions.zFieldNm == (self.tasktxt.text() + ' end time')
        if self.tasktxt.text()=='Walking': # for some reason, they switched the text to 'stop time' for walking
            SpQinds = self.questions.zFieldNm == (self.tasktxt.text() + ' stop time')
        SpQNb = self.questions[SpQinds].zFieldNb
        spTime = self.timestamps[sessInds][SpQNb].values[0][0]
        if type(spTime)!=str:
            print('No stop time found for subject ' + self.subjtxt.text()
                  + ' ' + self.sesstxt.text() + ' ' + self.tasktxt.text())
            return
        spTime = watchTime + self.getTimeDelta(spTime)
        
        if self.loadedSubject!=self.subjtxt.text():
            self.ACC = pd.read_csv(self.subjtxt.text() + '_Session1_Acc.csv', parse_dates=[3])
            self.loadedSubject = self.subjtxt.text()
        
        ACC = self.ACC
        ACC = ACC[(ACC.timestamp < (spTime + np.timedelta64(self.WindowRange,'s'))) 
                  & (ACC.timestamp > (stTime - np.timedelta64(self.WindowRange,'s')))]

        
        T = (ACC.timestamp.values.astype('datetime64[ms]') - ACC.timestamp.values[0].astype('datetime64[ms]')).astype(float)
        
        self.plot.clear()
        
        self.plot.plot(T,ACC.x.values,pen=(1,5))
        self.plot.plot(T,ACC.y.values,pen=(2,5))
        self.plot.plot(T,ACC.z.values,pen=(3,5))
        
        stTime = stTime.astype('datetime64[ms]').astype(float) - ACC.timestamp.values[0].astype('datetime64[ms]').astype(float)
        spTime = spTime.astype('datetime64[ms]').astype(float) - ACC.timestamp.values[0].astype('datetime64[ms]').astype(float)
        
        self.stLine.setValue(stTime); self.plot.addItem(self.stLine, ignoreBounds=True)
        self.spLine.setValue(spTime); self.plot.addItem(self.spLine, ignoreBounds=True)
        
        self.plot.setYRange(-3,3)
        
        
    def nextBtn_clicked(self):
        self.taskInd = self.taskInd + 1
        self.tasktxt.setText(self.taskList[self.taskInd])
        
        self.plotBtn_clicked()
        
    def saveBtn_clicked(self):
        stTime_ACC = self.ACC.timestamp.values[0].astype('datetime64[ms]')
        stTime = np.timedelta64(int(self.stLine.value()),'ms') + stTime_ACC
        spTime = np.timedelta64(int(self.spLine.value()),'ms') + stTime_ACC
        
        ACC = self.ACC
        ACC = ACC[(ACC.timestamp < (spTime)) & (ACC.timestamp > (stTime))]
        filepath = '//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\TaskAcc\\'+self.subjtxt.text()+'_'+self.tasktxt.text()+'.csv'
        ACC.to_csv(path_or_buf = filepath, index=False)
    
    def sessTxt_edited(self):
        self.sessionInd = np.argmax(self.sessionList == self.sesstxt.text())
        if sum(self.sessionList == self.sesstxt.text())<1:
            print('Not a valid session')
        
    def taskTxt_edited(self):
        self.taskInd = np.argmax(self.taskList == self.tasktxt.text())
        if sum(self.taskList == self.tasktxt.text())<1:
            print('Not a valid task')
    
    def getTimeDelta(self, ts): #Convert string stopwatch time to timedelta
        return np.timedelta64(int(ts[0:2]),'h') + np.timedelta64(int(ts[3:5]),'m') + np.timedelta64(int(ts[6:]),'s')
        
    

In [6]:
window = AccViewer()
window.show()
app.exec_()

Not a valid session
Not a valid session
Not a valid task
No start time found for subject 1004 2 Weeks: Time 0 Shaking


0